In [2]:
from yelpapi import YelpAPI
import numpy as np
import pandas as pd
import time
import pickle


In [3]:
#Load Yelp API key and connect to API

f = open('/home/henry/Insight/APIKey/YelpAPIKey.pckl', 'rb')
YelpAPIKey = pickle.load(f)
f.close()
yelpapi = YelpAPI(YelpAPIKey, timeout_s=3.0)


In [40]:
LandAreaFileLoc = "/home/henry/Insight/Yogee/Datasets/NY_Land_Area/NY_Land_Area.csv"
LandAreaDf = pd.read_csv(LandAreaFileLoc)

In [43]:
Zipcodes = LandAreaDf['GEO.id2']
SearchTerm = 'yoga'
SeachSort_by = 'distance'
SearchLimit = 50
SearchOffset = 0
YogaYelpRawDf = pd.DataFrame()


In [75]:
for i in range(0,np.size(Zipcodes)):
    SearchLocation = format(Zipcodes[i],'05d')
    SearchResults = yelpapi.search_query(term=SearchTerm, location=SearchLocation, sortby=SeachSort_by, limit=SearchLimit, offset=SearchOffset)
    time.sleep(.2)
    SearchDf = pd.DataFrame(SearchResults['businesses'])
    YogaYelpRawDf = pd.concat([YogaYelpDf, SearchDf],sort=False,ignore_index=True)

In [80]:
import pickle

f = open('/home/henry/Insight/Yogee/Datasets/Yelp_NY_Yoga_Studios_dataset/YogaYelpDfRAW.pckl', 'wb')
pickle.dump(YogaYelpRawDf, f)
f.close()

In [90]:
import pickle

f = open('/home/henry/Insight/Yogee/Datasets/Yelp_NY_Yoga_Studios_dataset/YogaYelpDfRAW.pckl', 'rb')
YogaYelpRawDf = pickle.load(f)
f.close()

In [108]:
# Drop duplicate stores

YogaReducedDf = YogaYelpRawDf.drop_duplicates(subset = 'id')

In [122]:
# Remove stores that do not have "yoga" in category

NotYoga = np.zeros(np.shape(YogaReducedDf)[0])
IndNotYoga = 0

for i in range(0,np.shape(YogaReducedDf)[0]):
    
    # Yoga category
    YogaCategory = False
    j = 0
    Categories = YogaReducedDf.iloc[i]['categories']
    while (not YogaCategory) and (j<len(Categories)):        
        alias = Categories[j]['alias']
        if alias == 'yoga':
            YogaCategory = True
        j = j + 1
        
    # Store location
    Address = YogaReducedDf.iloc[i]['location']
    State = Address['state']
    
    # Drop store index
    if (YogaCategory == False) or not(State == 'NY'):
        NotYoga[IndNotYoga] = YogaReducedDf.index[i]
        IndNotYoga = IndNotYoga + 1
        
NotYoga = pd.Int64Index(NotYoga[:IndNotYoga])
YogaYelpDf = YogaReducedDf.drop(NotYoga)
# Reindex Yoga studios dataframe

YogaYelpDf.index = pd.Int64Index(np.arange(0,np.shape(YogaYelpDf)[0]))

In [123]:
#Write Yelp yoga dataset to file
import pickle

f = open('/home/henry/Insight/Yogee/Datasets/Yelp_NY_Yoga_Studios_dataset/YogaYelpDf.pckl', 'wb')
pickle.dump(YogaYelpDf, f)
f.close()



In [129]:
# count studio number
NYcount = 0
NJcount = 0
PAcount = 0
VTcount = 0
MAcount = 0
CTcount = 0
States = []
for i in range(0,np.shape(YogaYelpDf)[0]):
    # Store location
    Address = YogaYelpDf.iloc[i]['location']
    State = Address['state']
    States.append(State)
    if State == 'NY':
        NYcount = NYcount + 1
    elif State == 'NJ':
        NJcount = NJcount + 1
    elif State == 'PA':
        PAcount = PAcount + 1
    elif State == 'VT':
        VTcount = VTcount + 1
    elif State == 'MA':
        MAcount = MAcount + 1
    elif State == 'CT':
        CTcount = CTcount + 1



In [130]:
NYcount

1229